## Introduction

In this tutorial, we will go through a complete QEC cycle.


In [8]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

In [9]:
def encode_gate():
    encoding_circuit = QuantumCircuit(7)
    encoding_circuit.h(0)
    encoding_circuit.h(4)
    encoding_circuit.h(6)
    encoding_circuit.cx(0, 1)
    encoding_circuit.cx(4, 5)
    encoding_circuit.cx(6, 3)
    encoding_circuit.cx(4, 2)
    encoding_circuit.cx(6, 5)
    encoding_circuit.cx(0, 3)
    encoding_circuit.cx(4, 1)
    encoding_circuit.cx(3, 2)

    return encoding_circuit.to_gate(label="$U_\\text{enc}$")

prev_syndrome = [0, 0, 0]
curr_syndrome = [0, 0, 0]

def first_flagged_syndrome_gate(iterations_remaining):
    l_qreg = QuantumRegister(7)
    a_qreg = QuantumRegister(3)
    m_creg = ClassicalRegister(3)
    qcirc = QuantumCircuit(l_qreg, a_qreg, m_creg)

    qcirc.reset(a_qreg)

    qcirc.h(a_qreg[0])

    qcirc.cx(a_qreg[0], l_qreg[3])
    qcirc.cx(l_qreg[2], a_qreg[2])
    qcirc.cx(l_qreg[5], a_qreg[1])
    qcirc.cx(a_qreg[0], a_qreg[1])
    qcirc.cx(a_qreg[0], l_qreg[0])
    qcirc.cx(l_qreg[3], a_qreg[2])
    qcirc.cx(l_qreg[4], a_qreg[1])
    qcirc.cx(a_qreg[0], l_qreg[1])
    qcirc.cx(l_qreg[6], a_qreg[2])
    qcirc.cx(l_qreg[2], a_qreg[1])
    qcirc.cx(a_qreg[0], a_qreg[2])
    qcirc.cx(a_qreg[0], l_qreg[2])
    qcirc.cx(l_qreg[5], a_qreg[2])
    qcirc.cx(l_qreg[1], a_qreg[1])

    qcirc.h(a_qreg[0])

    for i in range(3):
        with qcirc.if_test((m_creg[i], 0)) as else_:
            curr_syndrome[i] = 0
        with else_:
            curr_syndrome[i] = 1

    if prev_syndrome == curr_syndrome:
        qcirc = qcirc.compose(second_flagged_syndrome_gate(iterations_remaining), l_qreg[:] + a_qreg[:])
    else:
        qcirc = qcirc.compose(unflagged_syndrome_gate(iterations_remaining), l_qreg[:] + a_qreg[:])

    prev_syndrome = curr_syndrome

    return qcirc.to_gate(label="{$S_1^f, \\ S_5^f, \\ S_6^f$}")

def second_flagged_syndrome_gate(iterations_remaining):
    l_qreg = QuantumRegister(7)
    a_qreg = QuantumRegister(3)
    m_creg = ClassicalRegister(3)
    qcirc = QuantumCircuit(l_qreg, a_qreg, m_creg)

    qcirc.reset(a_qreg)

    qcirc.h(a_qreg[1])
    qcirc.h(a_qreg[2])

    qcirc.cx(l_qreg[3], a_qreg[0])
    qcirc.cx(a_qreg[2], l_qreg[2])
    qcirc.cx(a_qreg[1], l_qreg[5])
    qcirc.cx(a_qreg[1], a_qreg[0])
    qcirc.cx(l_qreg[0], a_qreg[0])
    qcirc.cx(a_qreg[2], l_qreg[3])
    qcirc.cx(a_qreg[1], l_qreg[4])
    qcirc.cx(l_qreg[1], a_qreg[0])
    qcirc.cx(a_qreg[2], l_qreg[6])
    qcirc.cx(a_qreg[1], l_qreg[2])
    qcirc.cx(a_qreg[2], a_qreg[0])
    qcirc.cx(l_qreg[2], a_qreg[0])
    qcirc.cx(a_qreg[2], l_qreg[5])
    qcirc.cx(a_qreg[1], l_qreg[1])

    qcirc.h(a_qreg[1])
    qcirc.h(a_qreg[2])

    for i in range(3):
        with qcirc.if_test((m_creg[i], 0)) as else_:
            curr_syndrome[i] = 0
        with else_:
            curr_syndrome[i] = 1

    if prev_syndrome == curr_syndrome:
        if iterations_remaining > 0:
            qcirc = qcirc.compose(first_flagged_syndrome_gate(iterations_remaining-1), l_qreg[:] + a_qreg[:])
    else:
        qcirc = qcirc.compose(unflagged_syndrome_gate(iterations_remaining), l_qreg[:] + a_qreg[:])

    prev_syndrome = curr_syndrome

    return qcirc.to_gate(label="{$S_2^f, \\ S_3^f, \\ S_4^f$}")

def unflagged_syndrome_gate(iterations_remaining):
    l_qreg = QuantumRegister(7)
    a_qreg = QuantumRegister(3)
    m_creg = ClassicalRegister(3)
    qcirc = QuantumCircuit(l_qreg, a_qreg, m_creg)

    qcirc.reset(a_qreg)
    
    qcirc.h(a_qreg[0])
    qcirc.cx(l_qreg[0],a_qreg[0])
    qcirc.cx(l_qreg[1],a_qreg[0])
    qcirc.cx(l_qreg[2],a_qreg[0])
    qcirc.cx(l_qreg[3],a_qreg[0])
    qcirc.h(a_qreg[0])

    qcirc.h(a_qreg[1])
    qcirc.cx(l_qreg[1],a_qreg[1])
    qcirc.cx(l_qreg[2],a_qreg[1])
    qcirc.cx(l_qreg[4],a_qreg[1])
    qcirc.cx(l_qreg[5],a_qreg[1])
    qcirc.h(a_qreg[1])

    qcirc.h(a_qreg[2])
    qcirc.cx(l_qreg[2],a_qreg[2])
    qcirc.cx(l_qreg[3],a_qreg[2])
    qcirc.cx(l_qreg[5],a_qreg[2])
    qcirc.cx(l_qreg[6],a_qreg[2])
    qcirc.h(a_qreg[2])

    for i in range(3):
        with qcirc.if_test((m_creg[i], 0)) as else_:
            curr_syndrome[i] = 0
        with else_:
            curr_syndrome[i] = 1

    if iterations_remaining > 0:
        qcirc = qcirc.compose(first_flagged_syndrome_gate(iterations_remaining-1), l_qreg[:] + a_qreg[:])

    prev_syndrome = curr_syndrome

    return qcirc.to_gate(label="{$S_1, S_2, S_3, S_4, S_5, S_6$}")

In [10]:
l_qreg = QuantumRegister(7)
a_qreg = QuantumRegister(3)
m_creg = ClassicalRegister(3)

qcirc = QuantumCircuit(l_qreg, a_qreg, m_creg)

In [11]:
qcirc.append(encode_gate(), l_qreg)

In [13]:
n_qec_cycles = 1
qcirc.append(first_flagged_syndrome_gate(n_qec_cycles))

UnboundLocalError: cannot access local variable 'prev_syndrome' where it is not associated with a value

In [ ]:
qcirc.draw("mpl")